In [16]:
import pandas as pd
import pyBigWig

data='/labs/mpsnyder/moqri/data/meth/'

v='38'
ezh_pk='https://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76626/suppl/GSE76626_ChIP-Seq_RPM.txt.gz' #hg38
h_bw='http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H9/tracks_hg'+v+'/Human_H9.meth.bw'

epic_f='https://ftp.ncbi.nlm.nih.gov/geo/series/GSE111nnn/GSE111165/suppl/GSE111165_data_processed_detection_p_val_EPIC.csv.gz'
epic_mf='https://webdata.illumina.com/downloads/productfiles/methylationEPIC/infinium-methylationepic-v-1-0-b5-manifest-file-csv.zip'
man450='https://zhouserver.research.chop.edu/InfiniumAnnotation/20180909/HM450/HM450.hg38.manifest.tsv.gz'

In [48]:
chr=6

In [18]:
h=pyBigWig.open(h_bw)

In [19]:
h1=h.intervals("chr"+str(ch))
h1=pd.DataFrame(h1)
h1=h1[h1[2]<.5]
h1=set(h1[1].tolist())
len(h1)

186247

In [93]:
ezh=pd.read_table(ezh_pk,index_col=0)
ezh=ezh[ezh.Chromosome!='MT']
ezh=ezh[['Chromosome','Start','End','WT_EZH2']][ezh['WT_EZH2']>1]
ezh.columns=['chrom', 'chromStart', 'chromEnd', 'value']
ezh['chrom']=ezh.chrom.astype(str)
ezh.shape

/home/moqri/anaconda3/envs/stem/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(3240, 4)

In [119]:
ezh[(ezh.chrom=='6')&(ezh.chromStart>37000000)]

,chrom,chromStart,chromEnd,value
Probe,,,,
MDGA1-003_upstream,6,37694311,37699311,2.534737
MDGA1-001_upstream,6,37695490,37700490,2.576863
MDGA1-007_upstream,6,37696806,37701806,2.287644
RP1-244F24.1-001_upstream,6,45419505,45424505,1.949774
RUNX2-009_upstream,6,45419536,45424536,1.949774
...,...,...,...,...
T-004_upstream,6,166166171,166171171,2.443764
T-002_upstream,6,166166200,166171200,2.443764
PRR18-001_upstream,6,166305883,166310883,1.652678


In [94]:
ezs={}
for ch in range(1,23):
    dfs=[]
    ez=ezh[ezh.chrom==str(ch)]
    ez['r']=ez.apply(lambda x:range(x['chromStart'],x['chromEnd']),1)
    ez=ez['r'].tolist()
    ez=set([item for sublist in ez for item in sublist])    
    ezs[ch]=ez

/home/moqri/anaconda3/envs/stem/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [95]:
len(ezs[chr])

265750

In [27]:
man=pd.read_table(data+'HM450.hg38.manifest.tsv.gz')

In [30]:
man.index=man.probeID

In [34]:
man=man[man.CpG_chrm=='chr6']
man.shape

(36613, 57)

In [38]:
man['c']=man.CpG_beg+1

In [125]:
37667555 in ezs[chr]

False

In [96]:
e1=man[man.c.isin(ezs[chr])&man.c.isin(h1)]
e1.shape

(902, 58)

In [63]:
cell='t'
df=pd.read_feather(data+cell+'.ft')
df=df.set_index('index')

In [97]:
ii=e1.index.intersection(df.columns)

In [99]:
df['m']=df[ii].median(1)

In [100]:
df[['m','age']].sort_values('age')

,m,age
index,,
100507.Mvalue,0.150409,45
101075.Mvalue,0.141114,45
100193.Mvalue,0.147032,45
101045.Mvalue,0.144775,45
100934.Mvalue,0.145781,45
...,...,...
101240.Mvalue,0.149940,76
100827.Mvalue,0.146967,77
100094.Mvalue,0.140396,78


In [104]:
df[ii].mean().sort_values()

cg24217844    0.028636
cg25211525    0.029868
cg26349375    0.031729
cg21530045    0.032862
cg10853637    0.032992
                ...   
cg09986316    0.825457
cg13280063    0.847725
cg07515565    0.872744
cg02037794    0.873141
cg04845615    0.888000
Length: 902, dtype: float64

In [90]:
df[['cg16867657','age']].sort_values('age')

,cg16867657,age
index,,
100507.Mvalue,0.669651,45
101075.Mvalue,0.661906,45
100193.Mvalue,0.640604,45
101045.Mvalue,0.613995,45
100934.Mvalue,0.660750,45
...,...,...
101240.Mvalue,0.757095,76
100827.Mvalue,0.787904,77
100094.Mvalue,0.746882,78


In [126]:
man.loc['cg04845615']

CpG_chrm                                                                 chr6
CpG_beg                                                            37699778.0
CpG_end                                                            37699780.0
probe_strand                                                                -
probeID                                                            cg04845615
address_A                                                            31775427
address_B                                                                 NaN
channel                                                                  Both
designType                                                                 II
nextBase                                                                  G/A
nextBaseRef                                                                 C
probeType                                                                  cg
orientation                                                     

In [ ]:
e=pd.read_csv(epic_mf,skiprows=7,index_col=0)
e=e[['CHR','Start_hg38','End_hg38']]
ch=6
len(ezs[ch-1])
e1=e[e.CHR==str(ch)]
e1['c']=e1.Start_hg38+1
e1.shape

In [2]:
df=pd.read_csv(epic_f,nrows=1000000,index_col=0)
df.index=df.ID_REF
df=df.drop('ID_REF',1)

<ipython-input-2-c6c431b17e73>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('ID_REF',1)


In [142]:
l=['201502810007_R02C01','200607090089_R08C01','200607090089_R01C01','201502850030_R04C01']
df[l].mean()

201502810007_R02C01    0.610467
200607090089_R08C01    0.630240
200607090089_R01C01    0.626962
201502850030_R04C01    0.625055
dtype: float64

In [143]:
df.loc['cg16867657'][l]

201502810007_R02C01    0.019754
200607090089_R08C01    0.017489
200607090089_R01C01    0.050277
201502850030_R04C01    0.194134
Name: cg16867657, dtype: float64

/home/moqri/anaconda3/envs/stem/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (2,4,42,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [79]:
e1=e1[e1.c.isin(ezs[ch-1])&e1.c.isin(h1)]
e1.shape

(943, 4)

In [80]:
ii=e1.index.intersection(df.index)

In [81]:
dfi=df.loc[ii]

In [135]:
df[l].mean()

201502800040_R05C01    0.620659
201502810041_R04C01    0.617868
201502810008_R03C01    0.625173
201502800040_R03C01    0.610210
200392810139_R08C01    0.630829
dtype: float64

In [144]:
dfi[l].mean()

201502810007_R02C01    0.111631
200607090089_R08C01    0.121581
200607090089_R01C01    0.126538
201502850030_R04C01    0.124365
dtype: float64

In [128]:
dfi.loc['cg16867657'][l]

201502810041_R04C01    0.325579
201502810008_R03C01    0.369649
201502800040_R03C01    0.533525
200392810139_R08C01    0.747467
Name: cg16867657, dtype: float64

In [91]:
dfi['m']=dfi[l].mean(1)

In [127]:
dfi=dfi.drop(dfi[dfi[[li+'_Detection_P_value' for li in l]].sum(1)>0].index)